## Baseline Submission: Toxic Language Classification 
**w207 Spring 2018 - Final Project Baseline**

**Team: Paul, Walt, Yisang, Joe**



### Project Description 

Our challenge is to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate.  The toxic language data set is sourced from Wikipedia and available as a public kaggle data set. 

Our goal is to use various machine learning techniques used in class to develop high quality ML models and pipelines.  

1. Exercise and build upon concepts covered in class and test out at least 3 kinds of supervised models:
    a. Regression (LASSO, Logistic)
    b. Trees (RF, XGBoost)
    c. DeepLearning (Tensorflow)
2. Using stacking/ensembling methods for improving prediction metrics (K-Means, anomaly detection)
3. Using unsupervised methods for feature engineering/selection

For the baseline proposal, this file contains a first pass run through from data preprocessing to model evaluation using a regression model pipeline. 

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge




### Data Ingestion

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

#scipy imports
from scipy.sparse import hstack

#Visualization imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import bokeh
#! pip install bokeh

#General imports
import pprint

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [7]:
# read frames localy through csv
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

# Random index generator for splitting training data
# Note: Each rerun of cell will create new splits.
randIndexCut = np.random.rand(len(train_df)) < 0.7

#S plit up data
test_data = test_df["comment_text"]
dev_data, dev_labels = train_df[~randIndexCut]["comment_text"], train_df[~randIndexCut][target_names]
train_data, train_labels = train_df[randIndexCut]["comment_text"], train_df[randIndexCut][target_names]

print 'total training observations:', train_df.shape[0]
print 'training data shape:', train_data.shape
print 'training label shape:', train_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', target_names

total training observations: 159571
training data shape: (111776,)
training label shape: (111776, 6)
dev label shape: (47795, 6)
labels names: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


### Exploratory Data Analysis

#### Class Imbalance

Let's see how imblanced the label set is in order to have a better understanding with the label quality of the given data set. 

In [8]:
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, output_file, output_notebook

target_counts = train_labels.apply(np.sum,0)
target_counts

output_notebook()


p = figure(x_range=target_names)
p.vbar(x=target_names, top = target_counts, width=0.9)

show(p)

train_labels.head()

Loading BokehJS ...

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,1,1,1,0,1,0


The data is fairly imbalanced when counting label occurrences. 

Ideas to consider
- Sampling methods
- Custom Cross Validation

### Feature Engineering/Selection (WIP)
....

### Modeling

### Text Processing

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
# Basic Count Vectorizer
countVector = CountVectorizer(ngram_range=(1,1))
train_counts = countVector.fit_transform(train_data)
dev_counts = countVector.fit_transform(dev_data)

print("\nVocabulary size is: {}").format(len(countVector.vocabulary_))
vocab_entries = {k: countVector.vocabulary_[k] for k in countVector.vocabulary_.keys()}
vocab_entries = pd.Series(vocab_entries).to_frame()
vocab_entries.columns = ['count']
vocab_entries = vocab_entries.sort_values(by='count')

print("Sample vocabulary from CountVectorizer:")
print(pp.pprint(vocab_entries.head(10)))
print("...")
print(pp.pprint(vocab_entries.tail(10)))
print("Number of nonzero entries in matrix: {}").format(train_counts.nnz)

tfidfVector = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
train_tfidf_counts = tfidfVector.fit_transform(train_data)
dev_tfidf_counts = tfidfVector.transform(dev_data)

print("\nVocabulary (tfidf) size is: {}").format(len(tfidfVector.vocabulary_))
vocab_entries = {k: tfidfVector.vocabulary_[k] for k in tfidfVector.vocabulary_.keys()}
vocab_entries = pd.Series(vocab_entries).to_frame()
vocab_entries.columns = ['count']
vocab_entries = vocab_entries.sort_values(by='count')

print("Sample vocabulary from TfidfVectorizer:")
print(pp.pprint(vocab_entries.head(10)))
print("...")
print(pp.pprint(vocab_entries.tail(10)))
print("Number of nonzero entries in matrix: {}").format(train_tfidf_counts.nnz)

#sample column wise sum, we can see that an observation can have multiple classes. 
count_df = pd.DataFrame(train_labels.apply(np.sum,1), columns = ["counts"])
count_df = count_df[((count_df["counts"] >= 1))]
count_df.head(10)


Vocabulary size is: 92035
Sample vocabulary from CountVectorizer:
           count
00             0
000            1
0000           2
000000         3
00000003       4
00000050       5
000001         6
000080         7
0001           8
000319821      9
None
...
      count
青い    92025
青山内科  92026
顧家輝   92027
길자비   92028
길잡이   92029
날짐승   92030
날찜승   92031
번역    92032
순혈주의  92033
유헌    92034
None
Number of nonzero entries in matrix: 4857719

Vocabulary (tfidf) size is: 152652
Sample vocabulary from TfidfVectorizer:
            count
00              0
000             1
0000            2
00000           3
000000          4
0000000         5
00000000        6
0000000027      7
00000001        8
000002          9
None
...
            count
칠지도        152642
편집         152643
ﬂute       152644
ａｎｏｎｔａｌｋ   152645
ｃｏｍ        152646
ｗｗｗ        152647
ｳｨｷﾍﾟﾃﾞｨｱ  152648
𐌰𐌹         152649
𐌰𐌿         152650
𐌴𐌹         152651
None
Number of nonzero entries in matrix: 2842221


,counts
6,4
12,1
16,1
42,4
43,3
44,1
51,2
55,4
59,1
65,3


### MLPClassifier (Neural Net)

In [ ]:
import time
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 

print("Modelling with MLPClassifier")

prediction_output = []
scores_output = []
full_CV_start = time.time()
for name in target_names:
    label_CV_start = time.time()
    classifier = MLPClassifier(hidden_layer_sizes=(6,6), activation='tanh', learning_rate='adaptive')
    classifier.fit(train_tfidf_counts, train_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, train_tfidf_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    label_CV_finish = time.time()
    print('Train data CV score for class {} is {}, after {} minutes.'.format(name, cv_score, 
                                                                                (label_CV_finish-label_CV_start)/60))
full_CV_finish = time.time()
print("Full cross-val across all labels took {} minutes.".format((full_CV_finish-full_CV_Start)/60))

print("Mean Train ROC_AUC for MLPClassifier: {}".format(np.mean(scores_output)))

prediction_output = []
scores_output = []
full_CV_start = time.time()
for name in target_names:
    label_CV_start = time.time()
    classifier = MLPClassifier(hidden_layer_sizes=(6,6), activation='tanh', learning_rate='adaptive')
    classifier.fit(dev_tfidf_counts, dev_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, dev_tfidf_counts, dev_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    label_CV_finish = time.time()
    print('DEV data CV score for class {} is {}, after {} minutes.'.format(name, cv_score, 
                                                                                (label_CV_finish-label_CV_start)/60))
full_CV_finish = time.time()
print("Full cross-val across all labels took {} minutes.".format(full_CV_finish-full_CV_Start))
print("Mean DEV ROC_AUC for MLPClassifier: {}".format(np.mean(scores_output)))
      


Modelling with MLPClassifier


### Keras (Neural Net with GPU support)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

print("Modelling with Keras")
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(dev_tfidf_counts, dev_labels):
  # create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# Fit the model
	model.fit(train_tfidf_counts, train_labels, epochs=150, batch_size=10, verbose=0)
	# evaluate the model
	scores = model.evaluate(dev_tfidf_counts, dev_labels, verbose=0)
	print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
	cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


### First Pass Logistic Regression with sag

In [ ]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

solver = 'sag'

print("Modelling with {} solver".format(solver))
prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver)
    classifier.fit(train_counts, train_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, train_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Training data CV score for class {} is {}'.format(name, cv_score))
    
print("Mean Training ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver) 
    classifier.fit(dev_counts, dev_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, dev_counts, dev_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Dev data CV score for class {} is {}'.format(name, cv_score))
        
print("Mean Dev ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))


### First Pass Logistic Regression with saga

In [ ]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

solver = 'saga'

print("Modelling with {} solver".format(solver))
prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver)
    classifier.fit(train_counts, train_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, train_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Training data CV score for class {} is {}'.format(name, cv_score))
    
print("Mean Training ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver) 
    classifier.fit(dev_counts, dev_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, dev_counts, dev_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Dev data CV score for class {} is {}'.format(name, cv_score))
        
print("Mean Dev ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))


### Here's the same using tfidf and saga

In [ ]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

solver = 'saga'

print("Modelling with {} solver".format(solver))
prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver)
    classifier.fit(train_tfidf_counts, train_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, train_tfidf_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Training data CV score for class {} is {}'.format(name, cv_score))

    
print("Mean Training ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver) 
    classifier.fit(dev_tfidf_counts, dev_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, dev_tfidf_counts, dev_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Dev data CV score for class {} is {}'.format(name, cv_score))

        
print("Mean Dev ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))


### Original counts with saga and L1

In [ ]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

solver = 'saga'

print("Modelling with {} solver".format(solver))
prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver,penalty='l1')
    classifier.fit(train_counts, train_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, train_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Training data CV score for class {} is {}'.format(name, cv_score))

    
print("Mean Training ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver,penalty='l1') 
    classifier.fit(dev_counts, dev_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, dev_counts, dev_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Dev data CV score for class {} is {}'.format(name, cv_score))

        
print("Mean Dev ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

### Tfidf with saga and L1

In [ ]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

solver = 'saga'

print("Modelling with {} solver".format(solver))
prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver,penalty='l1')
    classifier.fit(train_tfidf_counts, train_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, train_tfidf_counts, train_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Training data CV score for class {} is {}'.format(name, cv_score))

    
print("Mean Training ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

prediction_output = []
scores_output = []
for name in target_names:
    classifier = LogisticRegression(solver=solver,penalty='l1') 
    classifier.fit(dev_tfidf_counts, dev_labels[name])
    cv_score = np.mean(cross_val_score(
        classifier, dev_tfidf_counts, dev_labels[name], cv=3, scoring='roc_auc'))
    scores_output.append(cv_score)
    print('Dev data CV score for class {} is {}'.format(name, cv_score))

        
print("Mean Dev ROC_AUC for {} solver: {}").format(solver, np.mean(scores_output))

#### Testing on Dev Data

In [ ]:
from sklearn.metrics import auc, roc_curve
from sklearn import metrics

dev_Vector = CountVectorizer(ngram_range=(1,1))
dev_counts = countVector.fit_transform(dev_data)

pred_dt = pd.DataFrame()
scores_dev = []
for name in target_names:
    classifier = LogisticRegression(solver='sag') 
    classifier.fit(dev_counts, dev_labels[name])
    scores_dev.append(cv_score)
    output = classifier.predict(dev_counts)
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels[name], output)
    print('Dev score for class {} is {}'.format(name, metrics.auc(fpr,tpr)))
    pred_dt[name] = classifier.predict_proba(dev_counts)[:, 1]
    
    
print("Mean(dev) ROC_AUC: {}").format(np.mean(scores_dev))

Score on dev set is worse than training set, thus evidence of overfitting and a need for performance improvement.

The target is multi-label since each observation can be classified as multiple fields.  This is an important distinction from multi-class where each prediction can only be one label.  

## Evaluation

In [ ]:
count_df
train_labels["toxic"]

### Submission

In [ ]:
from sklearn.metrics import auc
# SK-learn libraries for cross validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
# Basic Logistic Regression Model/MultiLabel Edition

prediction_submission = pd.DataFrame()
prediction_submission["id"] = test_df["id"]

# new vector object for all train data for submission
finalTrainVector = CountVectorizer()
finalTrainCount = finalTrainVector.fit_transform(train_df["comment_text"])

# TODO: Using pipelines can clean up repetitive processes
# test set up
#testVector = CountVectorizer()
testCount = finalTrainVector.transform(test_df["comment_text"])

for name in target_names:
    classifier = LogisticRegression(solver='sag') #sag is one kind of solver optimize for multi-label
    clf = classifier.fit(finalTrainCount, train_df[name])
    prediction_submission[name] = clf.predict_proba(testCount)[:, 1]
    #print(prediction_submission)

    
print(prediction_submission.head(10)) # print frame output 
#prediction_submission.to_csv("submission.csv")

The frame contains the output for each class and is saved in a pandas data frame.  